In [2]:
import pandas as pd
import numpy as np

In [3]:
import requests
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# OpenAQ v3 — Fetch Delhi PM2.5 data with proper datetime handling
import requests
import pandas as pd
import time
from datetime import datetime
import json
import os
from pathlib import Path

API_KEY = os.environ.get("API_KEY")

HEADERS = {"X-API-Key": API_KEY, "Accept": "application/json"}

DATE_FROM = "2024-12-15T00:00:00Z"
DATE_TO   = "2025-02-14T23:59:59Z"

BASE = "https://api.openaq.org/v3"

def get_delhi_locations():
    """Get Delhi locations"""
    url = f"{BASE}/locations"
    r = requests.get(url, headers=HEADERS, params={"limit": 100}, timeout=30)
    r.raise_for_status()
    locations = r.json().get("results", [])

    delhi_locs = [
        loc for loc in locations
        if "delhi" in str(loc.get("name", "")).lower()
    ]
    return delhi_locs

def get_location_sensors(location_id):
    """Get all sensors for a location"""
    url = f"{BASE}/locations/{location_id}/sensors"
    r = requests.get(url, headers=HEADERS, params={"limit": 100}, timeout=30)
    if r.status_code == 200:
        return r.json().get("results", [])
    return []

def fetch_sensor_hours(sensor_id, date_from, date_to):
    """Fetch hourly data for a sensor"""
    url = f"{BASE}/sensors/{sensor_id}/hours"
    all_data = []
    page = 1
    max_pages = 30

    while page <= max_pages:
        params = {
            "date_from": date_from,
            "date_to": date_to,
            "limit": 1000,
            "page": page
        }

        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=30)
            if r.status_code == 404:
                break
            r.raise_for_status()

            results = r.json().get("results", [])

            # Debug: print first result structure
            if page == 1 and results and len(all_data) == 0:
                print(f"      Sample result structure: {json.dumps(results[0], indent=2)[:300]}")

            if not results:
                break

            all_data.extend(results)

            if len(results) < 1000:
                break

            page += 1
            time.sleep(0.2)

        except Exception as e:
            print(f"      Error: {e}")
            break

    return all_data

# ------------------------------
# Main pipeline
# ------------------------------

try:
    # Get Delhi locations
    print("\n1. Fetching Delhi locations...")
    locations = get_delhi_locations()
    print(f"   Found {len(locations)} Delhi locations\n")

    all_rows = []

    for idx, loc in enumerate(locations, 1):
        loc_id = loc.get("id")
        loc_name = loc.get("name")
        coords = loc.get("coordinates", {})
        lat = coords.get("latitude")
        lon = coords.get("longitude")

        print(f"{idx}. {loc_name} (ID: {loc_id})")

        # Get sensors for this location
        sensors = get_location_sensors(loc_id)

        if not sensors:
            print(f"- No sensors found")
            continue

        # Filter for PM2.5 sensors
        pm25_sensors = [
            s for s in sensors
            if s.get("parameter", {}).get("name", "").lower() == "pm25"
            or s.get("parameter", {}).get("id") == 2
        ]

        if not pm25_sensors:
            print(f"- Has {len(sensors)} sensors but no PM2.5")
            continue

        print(f"Found {len(pm25_sensors)} PM2.5 sensor(s)")

        for sensor in pm25_sensors:
            sensor_id = sensor.get("id")
            sensor_name = sensor.get("name")
            date_last = sensor.get("datetimeLast", {}).get("utc", "unknown")

            print(f"   - Sensor {sensor_id}: {sensor_name}")
            print(f"     Last data: {date_last}")

            # Check if sensor has data in our date range
            if date_last != "unknown":
                last_date = datetime.fromisoformat(date_last.replace("Z", "+00:00"))
                target_start = datetime.fromisoformat(DATE_FROM.replace("Z", "+00:00"))

                if last_date < target_start:
                    print(f"⚠ Sensor data too old, skipping")
                    continue

            # Fetch hourly data
            print(f"Fetching hourly data...")
            hours = fetch_sensor_hours(sensor_id, DATE_FROM, DATE_TO)

            if hours:
                print(f"     ✓ Got {len(hours)} hours")

                for h in hours:
                    # Extract datetime - try different field names
                    dt_utc = None
                    if "datetime" in h:
                        dt = h["datetime"]
                        if isinstance(dt, dict):
                            dt_utc = dt.get("utc")
                        else:
                            dt_utc = dt
                    elif "period" in h:
                        period = h["period"]
                        if isinstance(period, dict):
                            dt_utc = period.get("datetimeFrom", {}).get("utc") if isinstance(period.get("datetimeFrom"), dict) else period.get("datetimeFrom")

                    all_rows.append({
                        "location_id": loc_id,
                        "location_name": loc_name,
                        "latitude": lat,
                        "longitude": lon,
                        "sensor_id": sensor_id,
                        "datetime_utc": dt_utc,
                        "pm25": h.get("value"),
                        "unit": "µg/m³",
                        "coverage_percent": h.get("coverage", {}).get("percentComplete") if isinstance(h.get("coverage"), dict) else None
                    })
            else:
                print("No data in date range")

        time.sleep(0.3)

    print("\n" + "="*70)

    if all_rows:
        df = pd.DataFrame(all_rows)

        # Parse datetime
        df["datetime_utc"] = pd.to_datetime(df["datetime_utc"], utc=True, errors="coerce")

        # Remove rows with missing data
        df = df.dropna(subset=["pm25"])

        # Sort
        df = df.sort_values(["location_id", "sensor_id", "datetime_utc"]).reset_index(drop=True)

        # Create data folder if it doesn't exist
        data_folder = Path("data")
        data_folder.mkdir(exist_ok=True)

        # Save
        out_file = data_folder / "delhi_pm25_openaq_v3_final.csv"
        df.to_csv(out_file, index=False)

        print(f"\nSUCCESS! Saved {len(df)} PM2.5 records to '{out_file}'")
        print("="*70)
        print(f"\nData Summary:")
        print(f"   Shape: {df.shape}")
        print(f"   Date range: {df['datetime_utc'].min()} to {df['datetime_utc'].max()}")
        print(f"   Unique locations: {df['location_id'].nunique()}")
        print(f"   Unique sensors: {df['sensor_id'].nunique()}")

        print(f"\nPM2.5 Statistics (µg/m³):")
        stats = df['pm25'].describe()
        for stat_name, value in stats.items():
            print(f"   {stat_name:8s}: {value:,.2f}")

        print(f"\nSample Data (first 10 rows):")
        print(df.head(10)[['location_name', 'datetime_utc', 'pm25']].to_string())

        print(f"\nFull dataset saved to: {out_file}")

    else:
        print("\nNo PM2.5 data found for the specified date range")
        print(f"\nDate range requested: {DATE_FROM} to {DATE_TO}")

except Exception as e:
    print(f"\nError: {e}")
    import traceback
    traceback.print_exc()


1. Fetching Delhi locations...
   Found 4 Delhi locations

1. Delhi Technological University, Delhi - CPCB (ID: 13)
Found 1 PM2.5 sensor(s)
   - Sensor 13864: pm25 µg/m³
     Last data: 2018-02-22T04:00:00Z
⚠ Sensor data too old, skipping
2. R K Puram, Delhi - DPCC (ID: 17)
Found 2 PM2.5 sensor(s)
   - Sensor 35: pm25 µg/m³
     Last data: 2018-02-21T21:15:00Z
⚠ Sensor data too old, skipping
   - Sensor 12234787: pm25 µg/m³
     Last data: 2026-02-19T18:15:00Z
Fetching hourly data...
      Sample result structure: {
  "value": 110.0,
  "flagInfo": {
    "hasFlags": false
  },
  "parameter": {
    "id": 2,
    "name": "pm25",
    "units": "\u00b5g/m\u00b3",
    "displayName": null
  },
  "period": {
    "label": "1hour",
    "interval": "01:00:00",
    "datetimeFrom": {
      "utc": "2025-02-18T19:30:00Z",
   
     ✓ Got 7841 hours
3. Punjabi Bagh, Delhi - DPCC (ID: 50)
Found 2 PM2.5 sensor(s)
   - Sensor 396: pm25 µg/m³
     Last data: 2018-02-21T21:15:00Z
⚠ Sensor data too old, skipp

In [5]:
pm25df = pd.read_csv("data/delhi_pm25_openaq_v3_final.csv")
pm25df["datetime_hour"] = pd.to_datetime(pm25df["datetime_utc"]).dt.floor("h")
pm25df.head(20)

,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour
0,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 19:30:00+00:00,110.0,µg/m³,50.0,2025-02-18 19:00:00+00:00
1,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 20:30:00+00:00,94.8,µg/m³,100.0,2025-02-18 20:00:00+00:00
2,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 21:30:00+00:00,94.5,µg/m³,100.0,2025-02-18 21:00:00+00:00
3,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 22:30:00+00:00,102.0,µg/m³,100.0,2025-02-18 22:00:00+00:00
4,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 23:30:00+00:00,74.0,µg/m³,100.0,2025-02-18 23:00:00+00:00
5,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-19 00:30:00+00:00,97.8,µg/m³,100.0,2025-02-19 00:00:00+00:00
6,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-19 01:30:00+00:00,103.0,µg/m³,100.0,2025-02-19 01:00:00+00:00
7,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-19 02:30:00+00:00,97.5,µg/m³,100.0,2025-02-19 02:00:00+00:00
8,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-19 03:30:00+00:00,99.8,µg/m³,100.0,2025-02-19 03:00:00+00:00
9,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-19 04:30:00+00:00,80.5,µg/m³,100.0,2025-02-19 04:00:00+00:00


In [6]:
pm25df["location_name"].unique()

<StringArray>
['R K Puram, Delhi - DPCC', 'Punjabi Bagh, Delhi - DPCC']
Length: 2, dtype: str

In [7]:
# FIRMS Area API (uses MAP_KEY from env FIRMS_MAP_KEY)
# - Uses endpoint: /api/area/csv/[MAP_KEY]/[SOURCE]/[AREA_COORDINATES]/[DAY_RANGE]
# - AREA_COORDINATES = west,south,east,north (e.g. 74.0,27.0,79.5,31.5)
# - DAY_RANGE must be 1..5 (days)
# Replace bounding box or SOURCE as you like.

import os
import requests
import pandas as pd
from io import StringIO


MAP_KEY_ENVVAR = "FIRMS_MAP_KEY"   # you said you added this to env
SOURCE = "VIIRS_SNPP_NRT"          # Near-Real-Time VIIRS S-NPP (good for recent fires)
# Bounding box: west, south, east, north  (Delhi + Punjab/Haryana/UP slice)
AREA_COORDS = "74.0,27.0,79.5,31.5"
DAY_RANGE = 5                       # must be between 1 and 5 (inclusive)
OUT_CSV = "data/firms_area_fires_bbox.csv"
BASE_URL = "https://firms.modaps.eosdis.nasa.gov/api/area/csv"

# --------------------------
# Sanity checks
# --------------------------
map_key = os.environ.get(MAP_KEY_ENVVAR)
if not map_key:
    raise EnvironmentError(
        f"Environment variable '{MAP_KEY_ENVVAR}' not found. "
        "Set it to your FIRMS MAP_KEY and try again."
    )

if not (1 <= DAY_RANGE <= 5):
    raise ValueError("DAY_RANGE must be between 1 and 5 (inclusive).")

# Build request URL
# Format: /api/area/csv/[MAP_KEY]/[SOURCE]/[AREA_COORDINATES]/[DAY_RANGE]
url = f"{BASE_URL}/{map_key}/{SOURCE}/{AREA_COORDS}/{DAY_RANGE}"

print("Requesting FIRMS Area CSV:")
print(url)

# Fetch
resp = requests.get(url, timeout=60)
if resp.status_code != 200:
    # Helpful debugging info
    raise RuntimeError(
        f"FIRMS API request failed: HTTP {resp.status_code}\nResponse text:\n{resp.text}"
    )

# Parse CSV into pandas
csv_text = resp.text
df = pd.read_csv(StringIO(csv_text))

# Optional: keep only recommended columns
columns_to_keep = [
    "latitude",
    "longitude",
    "acq_date",
    "acq_time",
    "acq_datetime",     # some CSVs include this; if not, we'll create it
    "confidence",
    "frp",
    "brightness",
    "instrument",
    "satellite"
]
# Keep only available columns
cols_present = [c for c in columns_to_keep if c in df.columns]
df = df[cols_present].copy()

# Ensure acq_datetime column exists (create from acq_date + acq_time if needed)
if "acq_datetime" not in df.columns and {"acq_date", "acq_time"}.issubset(df.columns):
    df["acq_time"] = df["acq_time"].astype(str).str.zfill(4)  # ensure HHMM
    df["acq_datetime"] = pd.to_datetime(df["acq_date"] + " " + df["acq_time"], format="%Y-%m-%d %H%M", errors="coerce")

# Save cleaned CSV
df_fire = df
df_fire.to_csv(OUT_CSV, index=False)
print(f"FIRMS area data saved to '{OUT_CSV}' with {len(df)} rows.")
print("Columns saved:", df.columns.tolist())

df_fire["datetime_hour"] = pd.to_datetime(df_fire["acq_datetime"]).dt.floor("h")
df_fire.head(10)


Requesting FIRMS Area CSV:
https://firms.modaps.eosdis.nasa.gov/api/area/csv/fe917fc8c55db2e7396356bf5fbcb987/VIIRS_SNPP_NRT/74.0,27.0,79.5,31.5/5
FIRMS area data saved to 'data/firms_area_fires_bbox.csv' with 336 rows.
Columns saved: ['latitude', 'longitude', 'acq_date', 'acq_time', 'confidence', 'frp', 'instrument', 'satellite', 'acq_datetime']


,latitude,longitude,acq_date,acq_time,confidence,frp,instrument,satellite,acq_datetime,datetime_hour
0,27.00881,76.55927,2026-02-15,0802,n,2.15,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
1,27.17161,75.50048,2026-02-15,0802,l,5.74,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
2,27.17289,75.48715,2026-02-15,0802,l,3.63,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
3,27.17361,75.49136,2026-02-15,0802,n,4.91,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
4,27.17432,75.49555,2026-02-15,0802,l,4.91,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
5,27.71060,76.42234,2026-02-15,0802,n,2.77,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
6,27.71136,76.40573,2026-02-15,0802,n,2.70,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
7,27.82233,76.89230,2026-02-15,0802,n,3.40,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
8,27.83227,76.07059,2026-02-15,0802,n,6.51,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00
9,27.91287,76.21703,2026-02-15,0802,n,2.01,VIIRS,N,2026-02-15 08:02:00,2026-02-15 08:00:00


In [8]:
# aggregrating fire data per hour
fires_hourly = df_fire.groupby("datetime_hour").agg({
    "frp": "sum",
    "latitude": "count"
}).rename(columns={"latitude": "fire_count"}).reset_index()

In [9]:
stations = {
    "R K Puram": (28.5668, 77.1995),
    "Punjabi Bagh": (28.6715, 77.1234)
}


In [10]:
import requests
import pandas as pd

# Location: R K Puram (approx)
latitude = 28.5668
longitude = 77.1995

# Date range (adjust as needed)
start_date = "2026-02-10"
end_date = "2026-02-14"

# Open-Meteo API request
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": "wind_speed_10m,wind_direction_10m",
    "start_date": start_date,
    "end_date": end_date,
    "timezone": "auto"
}

# Make request and parse response
response = requests.get(url, params=params)
data = response.json()

# Create DataFrame
df = pd.DataFrame({
    "datetime": data["hourly"]["time"],
    "wind_speed_10m": data["hourly"]["wind_speed_10m"],
    "wind_direction_10m": data["hourly"]["wind_direction_10m"]
})
df["datetime"] = pd.to_datetime(df["datetime"])

# Save and preview
df.to_csv("data/rkpuram_wind_openmeteo.csv", index=False)
print("✅ Saved hourly wind data to 'rkpuram_wind_openmeteo.csv'")
df_weather_rk_puram = df
df_weather_rk_puram.head()
df_weather_rk_puram.columns

df_weather_rk_puram["datetime_hour"] = pd.to_datetime(df_weather_rk_puram["datetime"]).dt.floor("h")


✅ Saved hourly wind data to 'rkpuram_wind_openmeteo.csv'


In [11]:
# helper function to check if fire was in upwind direction of a station at a given time

In [12]:
import numpy as np

def calculate_bearing(lat1, lon1, lat2, lon2):
    # All in degrees
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dlon)
    bearing = np.degrees(np.arctan2(x, y))
    return (bearing + 360) % 360  # Normalize to 0–360

def is_fire_upwind(station_row, fire_row):
    fire_bearing = calculate_bearing(
        fire_row["latitude"], fire_row["longitude"],
        station_row["latitude"], station_row["longitude"]
    )
    wind_dir = station_row["wind_direction_10m"]
    # Wind blows *from* wind_dir; so reverse to get direction *toward* station
    wind_from = (wind_dir + 180) % 360
    # Check if fire bearing is roughly aligned with wind
    angle_diff = abs(wind_from - fire_bearing)
    return angle_diff <= 45  # Allow ±45° for tolerance


```python
# Fetch weather data for Punjabi Bagh
latitude_pb, longitude_pb = stations["Punjabi Bagh"]

url_pb = "https://api.open-meteo.com/v1/forecast"
params_pb = {
    "latitude": latitude_pb,
    "longitude": longitude_pb,
    "hourly": "wind_speed_10m,wind_direction_10m",
    "start_date": start_date,  # Use the same start_date as for R K Puram
    "end_date": end_date,      # Use the same end_date as for R K Puram
    "timezone": "auto"
}

response_pb = requests.get(url_pb, params=params_pb)
data_pb = response_pb.json()

df_weather_punjabi_bagh = pd.DataFrame({
    "datetime": data_pb["hourly"]["time"],
    "wind_speed_10m": data_pb["hourly"]["wind_speed_10m"],
    "wind_direction_10m": data_pb["hourly"]["wind_direction_10m"]
})
df_weather_punjabi_bagh["datetime"] = pd.to_datetime(df_weather_punjabi_bagh["datetime"])
df_weather_punjabi_bagh["datetime_hour"] = df_weather_punjabi_bagh["datetime"].dt.floor("h")

print("✅ Fetched hourly wind data for Punjabi Bagh.")
df_weather_punjabi_bagh.head()
```

In [13]:
latitude_pb = stations["Punjabi Bagh"][0]
longitude_pb = stations["Punjabi Bagh"][1]

url_pb = "https://api.open-meteo.com/v1/forecast"
params_pb = {
    "latitude": latitude_pb,
    "longitude": longitude_pb,
    "hourly": "wind_speed_10m,wind_direction_10m",
    "start_date": start_date,
    "end_date": end_date,
    "timezone": "auto"
}

In [14]:
response_pb = requests.get(url_pb, params=params_pb)

In [15]:
data_pb = response_pb.json()

In [16]:
df_weather_punjabi_bagh = pd.DataFrame({
    "datetime": data_pb["hourly"]["time"],
    "wind_speed_10m": data_pb["hourly"]["wind_speed_10m"],
    "wind_direction_10m": data_pb["hourly"]["wind_direction_10m"]
})
df_weather_punjabi_bagh["datetime"] = pd.to_datetime(df_weather_punjabi_bagh["datetime"])
df_weather_punjabi_bagh["datetime_hour"] = df_weather_punjabi_bagh["datetime"].dt.floor("h")

print("✅ Fetched hourly wind data for Punjabi Bagh.")
df_weather_punjabi_bagh.head()

✅ Fetched hourly wind data for Punjabi Bagh.


,datetime,wind_speed_10m,wind_direction_10m,datetime_hour
0,2026-02-10 00:00:00,4.2,110,2026-02-10 00:00:00
1,2026-02-10 01:00:00,4.6,108,2026-02-10 01:00:00
2,2026-02-10 02:00:00,4.7,94,2026-02-10 02:00:00
3,2026-02-10 03:00:00,4.7,90,2026-02-10 03:00:00
4,2026-02-10 04:00:00,4.5,104,2026-02-10 04:00:00


In [17]:
df_weather_rk_puram["location_name"] = "R K Puram, Delhi - DPCC"

In [18]:
df_weather_punjabi_bagh["location_name"] = "Punjabi Bagh, Delhi - DPCC"

In [19]:
df_weather = pd.concat([df_weather_rk_puram, df_weather_punjabi_bagh], ignore_index=True)

In [20]:
df_weather

,datetime,wind_speed_10m,wind_direction_10m,datetime_hour,location_name
0,2026-02-10 00:00:00,2.9,120,2026-02-10 00:00:00,"R K Puram, Delhi - DPCC"
1,2026-02-10 01:00:00,3.2,117,2026-02-10 01:00:00,"R K Puram, Delhi - DPCC"
2,2026-02-10 02:00:00,3.7,101,2026-02-10 02:00:00,"R K Puram, Delhi - DPCC"
3,2026-02-10 03:00:00,4.3,90,2026-02-10 03:00:00,"R K Puram, Delhi - DPCC"
4,2026-02-10 04:00:00,3.9,112,2026-02-10 04:00:00,"R K Puram, Delhi - DPCC"
...,...,...,...,...,...
235,2026-02-14 19:00:00,3.1,234,2026-02-14 19:00:00,"Punjabi Bagh, Delhi - DPCC"
236,2026-02-14 20:00:00,4.3,228,2026-02-14 20:00:00,"Punjabi Bagh, Delhi - DPCC"
237,2026-02-14 21:00:00,3.7,241,2026-02-14 21:00:00,"Punjabi Bagh, Delhi - DPCC"
238,2026-02-14 22:00:00,3.3,276,2026-02-14 22:00:00,"Punjabi Bagh, Delhi - DPCC"


In [21]:
if df_weather['datetime_hour'].dt.tz is None:
    df_weather['datetime_hour'] = df_weather['datetime_hour'].dt.tz_localize('UTC')
df_merged = pd.merge(pm25df, df_weather, how="left",
                     on=["datetime_hour", "location_name"])

df_merged.head(5)

,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,datetime,wind_speed_10m,wind_direction_10m
0,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 19:30:00+00:00,110.0,µg/m³,50.0,2025-02-18 19:00:00+00:00,NaT,NaN,NaN
1,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 20:30:00+00:00,94.8,µg/m³,100.0,2025-02-18 20:00:00+00:00,NaT,NaN,NaN
2,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 21:30:00+00:00,94.5,µg/m³,100.0,2025-02-18 21:00:00+00:00,NaT,NaN,NaN
3,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 22:30:00+00:00,102.0,µg/m³,100.0,2025-02-18 22:00:00+00:00,NaT,NaN,NaN
4,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 23:30:00+00:00,74.0,µg/m³,100.0,2025-02-18 23:00:00+00:00,NaT,NaN,NaN


In [22]:
print(f"pm25df date range: {pm25df['datetime_hour'].min()} to {pm25df['datetime_hour'].max()}")
print(f"df_weather date range: {df_weather['datetime_hour'].min()} to {df_weather['datetime_hour'].max()}")


pm25df date range: 2025-02-18 19:00:00+00:00 to 2026-02-19 17:00:00+00:00
df_weather date range: 2026-02-10 00:00:00+00:00 to 2026-02-14 23:00:00+00:00


In [23]:
df_merged_success = df_merged.dropna(subset=['datetime'])
if not df_merged_success.empty:
    display(df_merged_success.head())
else:
    print("No overlapping data found based on the current date ranges.")

df_merged_success


,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,datetime,wind_speed_10m,wind_direction_10m
7659,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 00:30:00+00:00,154.0,µg/m³,100.0,2026-02-10 00:00:00+00:00,2026-02-10 00:00:00,2.9,120.0
7660,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 01:30:00+00:00,185.0,µg/m³,100.0,2026-02-10 01:00:00+00:00,2026-02-10 01:00:00,3.2,117.0
7661,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 02:30:00+00:00,199.0,µg/m³,100.0,2026-02-10 02:00:00+00:00,2026-02-10 02:00:00,3.7,101.0
7662,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 03:30:00+00:00,182.0,µg/m³,100.0,2026-02-10 03:00:00+00:00,2026-02-10 03:00:00,4.3,90.0
7663,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 04:30:00+00:00,118.0,µg/m³,100.0,2026-02-10 04:00:00+00:00,2026-02-10 04:00:00,3.9,112.0


,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,datetime,wind_speed_10m,wind_direction_10m
7659,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 00:30:00+00:00,154.0,µg/m³,100.0,2026-02-10 00:00:00+00:00,2026-02-10 00:00:00,2.9,120.0
7660,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 01:30:00+00:00,185.0,µg/m³,100.0,2026-02-10 01:00:00+00:00,2026-02-10 01:00:00,3.2,117.0
7661,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 02:30:00+00:00,199.0,µg/m³,100.0,2026-02-10 02:00:00+00:00,2026-02-10 02:00:00,3.7,101.0
7662,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 03:30:00+00:00,182.0,µg/m³,100.0,2026-02-10 03:00:00+00:00,2026-02-10 03:00:00,4.3,90.0
7663,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 04:30:00+00:00,118.0,µg/m³,100.0,2026-02-10 04:00:00+00:00,2026-02-10 04:00:00,3.9,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15456,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-14 19:30:00+00:00,151.0,µg/m³,100.0,2026-02-14 19:00:00+00:00,2026-02-14 19:00:00,3.1,234.0
15457,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-14 20:30:00+00:00,168.0,µg/m³,100.0,2026-02-14 20:00:00+00:00,2026-02-14 20:00:00,4.3,228.0
15458,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-14 21:30:00+00:00,149.0,µg/m³,100.0,2026-02-14 21:00:00+00:00,2026-02-14 21:00:00,3.7,241.0
15459,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-14 22:30:00+00:00,121.0,µg/m³,100.0,2026-02-14 22:00:00+00:00,2026-02-14 22:00:00,3.3,276.0


In [24]:
df_fire['datetime_hour'] = df_fire['datetime_hour'].dt.tz_localize('UTC')
df_merged_success = pd.merge(df_merged_success, df_fire, how="left", on="datetime_hour")

# Task
Convert the `datetime_hour` column in `df_fire` to be timezone-aware (UTC) to match `df_merged_success['datetime_hour']` before performing the merge operation.

## Fix Datetime Type Mismatch for Merging

### Subtask:
Convert the `datetime_hour` column in `df_fire` to be timezone-aware (UTC) to match `df_merged_success['datetime_hour']` before performing the merge operation.


## Summary:

### Data Analysis Key Findings
- The `datetime_hour` column in `df_fire` was successfully converted to a timezone-aware datetime format, specifically set to UTC.
- This conversion ensures that `df_fire['datetime_hour']` now matches the timezone-aware UTC format of `df_merged_success['datetime_hour']`.

### Insights or Next Steps
- This data preparation step is critical for performing accurate and consistent merge operations between the `df_fire` and `df_merged_success` DataFrames.
- The next logical step is to proceed with merging these DataFrames on the now compatible `datetime_hour` column.


In [25]:
df_fire['datetime_hour'] = df_fire['datetime_hour'].dt.tz_convert('UTC')
df_merged_success = pd.merge(df_merged_success, df_fire, how="left", on="datetime_hour")
display(df_merged_success.head())

,location_id,location_name,latitude_x,longitude_x,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,...,acq_datetime_x,latitude,longitude,acq_date_y,acq_time_y,confidence_y,frp_y,instrument_y,satellite_y,acq_datetime_y
0,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 00:30:00+00:00,154.0,µg/m³,100.0,2026-02-10 00:00:00+00:00,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 01:30:00+00:00,185.0,µg/m³,100.0,2026-02-10 01:00:00+00:00,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 02:30:00+00:00,199.0,µg/m³,100.0,2026-02-10 02:00:00+00:00,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 03:30:00+00:00,182.0,µg/m³,100.0,2026-02-10 03:00:00+00:00,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-10 04:30:00+00:00,118.0,µg/m³,100.0,2026-02-10 04:00:00+00:00,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


# Task
The date ranges used for fetching weather data are not aligned with the PM2.5 and fire data, which is preventing a successful merge. To fix this, I'll update the `start_date` and `end_date` variables in cell `833539fa` to "2026-02-15" and "2026-02-18" respectively. After that, I'll re-run all the subsequent cells to apply these changes and properly merge the datasets.

## Update weather fetch date range

### Subtask:
Adjust the `start_date` and `end_date` variables in cell `833539fa` to ensure an overlapping date range with the fire data.


**Reasoning**:
The subtask requires modifying the `start_date` and `end_date` variables in cell `833539fa` to ensure an overlapping date range with the fire data. I will update these variables in the specified cell.



In [26]:
import requests
import pandas as pd

# Location: R K Puram (approx)
latitude = 28.5668
longitude = 77.1995

# Date range (adjust as needed)
start_date = "2026-02-15"
end_date = "2026-02-18"

# Open-Meteo API request
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": "wind_speed_10m,wind_direction_10m",
    "start_date": start_date,
    "end_date": end_date,
    "timezone": "auto"
}

# Make request and parse response
response = requests.get(url, params=params)
data = response.json()

# Create DataFrame
df = pd.DataFrame({
    "datetime": data["hourly"]["time"],
    "wind_speed_10m": data["hourly"]["wind_speed_10m"],
    "wind_direction_10m": data["hourly"]["wind_direction_10m"]
})
df["datetime"] = pd.to_datetime(df["datetime"])

# Save and preview
df.to_csv("data/rkpuram_wind_openmeteo.csv", index=False)
print("✅ Saved hourly wind data to 'rkpuram_wind_openmeteo.csv'")
df_weather_rk_puram = df
df_weather_rk_puram.head()
df_weather_rk_puram.columns

df_weather_rk_puram["datetime_hour"] = pd.to_datetime(df_weather_rk_puram["datetime"]).dt.floor("h")

✅ Saved hourly wind data to 'rkpuram_wind_openmeteo.csv'


**Reasoning**:
Now that the R K Puram weather data has been refetched with the correct date range, I need to refetch the Punjabi Bagh weather data with the same updated date range to ensure consistency before merging.



In [27]:
latitude_pb = stations["Punjabi Bagh"][0]
longitude_pb = stations["Punjabi Bagh"][1]

url_pb = "https://api.open-meteo.com/v1/forecast"
params_pb = {
    "latitude": latitude_pb,
    "longitude": longitude_pb,
    "hourly": "wind_speed_10m,wind_direction_10m",
    "start_date": start_date,
    "end_date": end_date,
    "timezone": "auto"
}

**Reasoning**:
I need to complete the API request for Punjabi Bagh weather data and process the response to update `df_weather_punjabi_bagh` using the previously defined `start_date` and `end_date`.



In [28]:
latitude_pb = stations["Punjabi Bagh"][0]
longitude_pb = stations["Punjabi Bagh"][1]

url_pb = "https://api.open-meteo.com/v1/forecast"
params_pb = {
    "latitude": latitude_pb,
    "longitude": longitude_pb,
    "hourly": "wind_speed_10m,wind_direction_10m",
    "start_date": start_date,
    "end_date": end_date,
    "timezone": "auto"
}

response_pb = requests.get(url_pb, params=params_pb)
data_pb = response_pb.json()

df_weather_punjabi_bagh = pd.DataFrame({
    "datetime": data_pb["hourly"]["time"],
    "wind_speed_10m": data_pb["hourly"]["wind_speed_10m"],
    "wind_direction_10m": data_pb["hourly"]["wind_direction_10m"]
})
df_weather_punjabi_bagh["datetime"] = pd.to_datetime(df_weather_punjabi_bagh["datetime"])
df_weather_punjabi_bagh["datetime_hour"] = df_weather_punjabi_bagh["datetime"].dt.floor("h")

print("✅ Fetched hourly wind data for Punjabi Bagh.")
df_weather_punjabi_bagh.head()

✅ Fetched hourly wind data for Punjabi Bagh.


,datetime,wind_speed_10m,wind_direction_10m,datetime_hour
0,2026-02-15 00:00:00,3.1,306,2026-02-15 00:00:00
1,2026-02-15 01:00:00,2.6,304,2026-02-15 01:00:00
2,2026-02-15 02:00:00,2.4,297,2026-02-15 02:00:00
3,2026-02-15 03:00:00,3.2,270,2026-02-15 03:00:00
4,2026-02-15 04:00:00,4.8,257,2026-02-15 04:00:00


**Reasoning**:
Now that both R K Puram and Punjabi Bagh weather data have been refetched with the correct date ranges, I need to assign the 'location_name' column to the `df_weather_rk_puram` DataFrame to prepare it for concatenation with `df_weather_punjabi_bagh`.



In [29]:
df_weather_rk_puram["location_name"] = "R K Puram, Delhi - DPCC"

**Reasoning**:
Following the assignment of 'location_name' to `df_weather_rk_puram`, I need to assign the corresponding 'location_name' to the `df_weather_punjabi_bagh` DataFrame to ensure consistency before concatenating the weather dataframes.



In [30]:
df_weather_punjabi_bagh["location_name"] = "Punjabi Bagh, Delhi - DPCC"

**Reasoning**:
With both `df_weather_rk_puram` and `df_weather_punjabi_bagh` now having their respective 'location_name' columns, I need to concatenate them into a single `df_weather` DataFrame to combine all weather data, ensuring all necessary weather data is merged for subsequent analysis.



In [31]:
df_weather = pd.concat([df_weather_rk_puram, df_weather_punjabi_bagh], ignore_index=True)

**Reasoning**:
With both `df_weather_rk_puram` and `df_weather_punjabi_bagh` now having their respective 'location_name' columns, I need to concatenate them into a single `df_weather` DataFrame to combine all weather data, ensuring all necessary weather data is merged for subsequent analysis.



In [32]:
df_weather = pd.concat([df_weather_rk_puram, df_weather_punjabi_bagh], ignore_index=True)

**Reasoning**:
The weather dataframes `df_weather_rk_puram` and `df_weather_punjabi_bagh` have now been updated and their 'location_name' columns assigned. I will now concatenate these two dataframes into a single `df_weather` DataFrame.



In [33]:
df_weather = pd.concat([df_weather_rk_puram, df_weather_punjabi_bagh], ignore_index=True)

**Reasoning**:
The weather dataframes `df_weather_rk_puram` and `df_weather_punjabi_bagh` have now been updated and their 'location_name' columns assigned. I will now concatenate these two dataframes into a single `df_weather` DataFrame.



In [34]:
df_weather = pd.concat([df_weather_rk_puram, df_weather_punjabi_bagh], ignore_index=True)

**Reasoning**:
The weather dataframes for both locations have been updated and concatenated into `df_weather`. Now, to ensure compatibility for merging, I need to make the `datetime_hour` column in the newly created `df_weather` DataFrame timezone-aware (UTC) to match the `datetime_hour` in `pm25df` and `df_fire`.



In [35]:
if df_weather['datetime_hour'].dt.tz is None:
    df_weather['datetime_hour'] = df_weather['datetime_hour'].dt.tz_localize('UTC')
df_merged = pd.merge(pm25df, df_weather, how="left",
                     on=["datetime_hour", "location_name"])

df_merged.head(5)

,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,datetime,wind_speed_10m,wind_direction_10m
0,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 19:30:00+00:00,110.0,µg/m³,50.0,2025-02-18 19:00:00+00:00,NaT,NaN,NaN
1,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 20:30:00+00:00,94.8,µg/m³,100.0,2025-02-18 20:00:00+00:00,NaT,NaN,NaN
2,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 21:30:00+00:00,94.5,µg/m³,100.0,2025-02-18 21:00:00+00:00,NaT,NaN,NaN
3,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 22:30:00+00:00,102.0,µg/m³,100.0,2025-02-18 22:00:00+00:00,NaT,NaN,NaN
4,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2025-02-18 23:30:00+00:00,74.0,µg/m³,100.0,2025-02-18 23:00:00+00:00,NaT,NaN,NaN


**Reasoning**:
After merging the `pm25df` and `df_weather` DataFrames into `df_merged`, there are expected `NaN` values where the date ranges do not overlap. To proceed with the analysis of successfully merged data, I need to remove these rows by dropping entries where the `datetime` column from the weather data is null. This will create a `df_merged_success` DataFrame containing only the records with corresponding weather information.



In [36]:
df_merged_success = df_merged.dropna(subset=['datetime'])
if not df_merged_success.empty:
    display(df_merged_success.head())
else:
    print("No overlapping data found based on the current date ranges.")

df_merged_success

,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,datetime,wind_speed_10m,wind_direction_10m
7778,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 00:30:00+00:00,83.0,µg/m³,100.0,2026-02-15 00:00:00+00:00,2026-02-15 00:00:00,4.8,297.0
7779,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 01:30:00+00:00,92.0,µg/m³,100.0,2026-02-15 01:00:00+00:00,2026-02-15 01:00:00,4.5,299.0
7780,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 02:30:00+00:00,103.0,µg/m³,100.0,2026-02-15 02:00:00+00:00,2026-02-15 02:00:00,4.6,288.0
7781,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 03:30:00+00:00,105.0,µg/m³,100.0,2026-02-15 03:00:00+00:00,2026-02-15 03:00:00,4.4,279.0
7782,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 04:30:00+00:00,95.8,µg/m³,100.0,2026-02-15 04:00:00+00:00,2026-02-15 04:00:00,5.4,270.0


,location_id,location_name,latitude,longitude,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,datetime,wind_speed_10m,wind_direction_10m
7778,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 00:30:00+00:00,83.0,µg/m³,100.0,2026-02-15 00:00:00+00:00,2026-02-15 00:00:00,4.8,297.0
7779,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 01:30:00+00:00,92.0,µg/m³,100.0,2026-02-15 01:00:00+00:00,2026-02-15 01:00:00,4.5,299.0
7780,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 02:30:00+00:00,103.0,µg/m³,100.0,2026-02-15 02:00:00+00:00,2026-02-15 02:00:00,4.6,288.0
7781,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 03:30:00+00:00,105.0,µg/m³,100.0,2026-02-15 03:00:00+00:00,2026-02-15 03:00:00,4.4,279.0
7782,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 04:30:00+00:00,95.8,µg/m³,100.0,2026-02-15 04:00:00+00:00,2026-02-15 04:00:00,5.4,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15501,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-18 11:30:00+00:00,37.8,µg/m³,100.0,2026-02-18 11:00:00+00:00,2026-02-18 11:00:00,3.9,68.0
15502,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-18 12:30:00+00:00,55.0,µg/m³,100.0,2026-02-18 12:00:00+00:00,2026-02-18 12:00:00,4.0,100.0
15503,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-18 13:30:00+00:00,58.8,µg/m³,100.0,2026-02-18 13:00:00+00:00,2026-02-18 13:00:00,5.2,102.0
15504,50,"Punjabi Bagh, Delhi - DPCC",28.674045,77.131023,12234796,2026-02-18 14:30:00+00:00,88.3,µg/m³,100.0,2026-02-18 14:00:00+00:00,2026-02-18 14:00:00,5.5,122.0


**Reasoning**:
Now that the `df_merged_success` DataFrame contains PM2.5 and weather data for overlapping time ranges, I need to merge it with the `df_fire` DataFrame to incorporate fire incident data. This merge will be performed on the common 'datetime_hour' column, which is now timezone-aware in both dataframes.



In [37]:
df_merged_success = pd.merge(df_merged_success, df_fire, how="left", on="datetime_hour")
display(df_merged_success.head())

,location_id,location_name,latitude_x,longitude_x,sensor_id,datetime_utc,pm25,unit,coverage_percent,datetime_hour,...,wind_direction_10m,latitude_y,longitude_y,acq_date,acq_time,confidence,frp,instrument,satellite,acq_datetime
0,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 00:30:00+00:00,83.0,µg/m³,100.0,2026-02-15 00:00:00+00:00,...,297.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 01:30:00+00:00,92.0,µg/m³,100.0,2026-02-15 01:00:00+00:00,...,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 02:30:00+00:00,103.0,µg/m³,100.0,2026-02-15 02:00:00+00:00,...,288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 03:30:00+00:00,105.0,µg/m³,100.0,2026-02-15 03:00:00+00:00,...,279.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,17,"R K Puram, Delhi - DPCC",28.563262,77.186937,12234787,2026-02-15 04:30:00+00:00,95.8,µg/m³,100.0,2026-02-15 04:00:00+00:00,...,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
